In [1]:
#Gathering info from a user
print("Design a Quiz")
print("Which book would you like to use?")
print ("Here are your choices:\n")
print("The Best Bad Luck I Ever Had \n The Lions of Little Rock\n The Paper Cowboy\n The Thing I'm Most Afraid Of\n Anne of Green Gables")
book = input()
if book == "The Best Bad Luck I Ever Had":
    narrator = "Dit"
    text = "Book_1"
elif book == "The Lions of Little Rock":
    narrator = "Marlee"
    text = "Book_2"
elif book == "The Paper Cowboy":
    narrator = "Tommy"
    text = "Book_3"
elif book == "The Thing I'm Most Afraid Of":
    narrator = "Becca"
    text = "Book_4"
elif book == "Anne of Green Gables":
    narrator = ''
    text = "Book_5"
else:
    print("I'm sorry. That book is not in our database.")

Design a Quiz
Which book would you like to use?
Here are your choices:

The Best Bad Luck I Ever Had 
 The Lions of Little Rock
 The Paper Cowboy
 The Thing I'm Most Afraid Of
 Anne of Green Gables
The Best Bad Luck I Ever Had


In [2]:
#Creating a file dictionary
#This assumes the files are broken down into chapters in a folder
import os
import nltk
import spacy
import pandas as pd
nlp = spacy.load("en")

#Creating the file dictionary
# point this to the data directory
direc = "/Users/kristinlevine/Desktop/NLP_Project/" + text 
ext = '.txt' # Select your file delimiter

file_dict = {} # Create an empty dict

# Select only files with the ext extension
txt_files = [i for i in os.listdir(direc) if os.path.splitext(i)[1] == ext]

# Iterate over your txt files
for f in txt_files:
    # Open them and assign them to file_dict
    with open(os.path.join(direc,f)) as file_object:
        file_dict[f] = file_object.read()

In [3]:
print("There are", len(txt_files), 'chapters in this book.')
print("Which chapter would you like to analyze?")
chapter = input()

There are 62 chapters in this book.
Which chapter would you like to analyze?
33


In [4]:
#To extract 
def extract_entities(doc_id, doc_text):
    analyzed_doc = nlp(doc_text)

    doc_persons = {}
    
    for entity in analyzed_doc.ents:
        if entity.text.strip() != "" and entity.label_ == "PERSON":
            
            if entity.text.strip() not in doc_persons.keys():
                relevant_sentence = (doc_id, entity.sent.text)
                doc_persons[entity.text.strip()] = list()
                
            if entity.text.strip() in doc_persons.keys():
                relevant_sentence = (doc_id, entity.sent.text)
                doc_persons[entity.text.strip()].append(relevant_sentence)  
            
    return doc_persons

In [5]:
def find_most_popular_entities(entity_dictionary):
    
    list_most_mentions = {}
    
    for entity in entity_dictionary:
        x = []
        for i in range(len(entity_dictionary[entity])):
            x.append(len(entity_dictionary[entity][i]))
        list_most_mentions[entity] = sum(x)
        
    # sort through the entities in the dictionary by the number of sentences
    
    return list_most_mentions

In [6]:
q = extract_entities(chapter + '.txt', file_dict[chapter +'.txt'])
tp = find_most_popular_entities(q)

In [7]:
#Persons
chapter_persons = pd.DataFrame.from_dict(tp, orient = 'index')
chapter_persons = chapter_persons.rename(columns = {0:'Count'})
chapter_persons = chapter_persons.sort_values(by=['Count'], ascending = False)
#print(chapter_persons)

In [8]:
combined_persons = {}

for item in txt_files: 
    persons = extract_entities(item, file_dict[item])

#For Persons
    for per in persons.keys():
        if per not in combined_persons.keys():
            combined_persons[per] = list()
        
        if per in combined_persons.keys():
            combined_persons[per].append(persons.get(per))

In [9]:
#Creating a List of the top 20 characters in the book
all_characters = find_most_popular_entities(combined_persons)
char_in_book = pd.DataFrame.from_dict(all_characters, orient = 'index')
char_in_book = char_in_book.rename(columns = {0:"Count"})
char_in_book = char_in_book.sort_values(by=['Count'], ascending = False)
main_char = char_in_book.head(21)
people = main_char.index.tolist()
#We also need to remove the narrator from this list because he/she can't talk to him/herself.
if narrator in people:
    people.remove(narrator)
else: 
    pass
#print(people)

In [10]:
#Check the characters in each chapters against the main_character (people) list.
#This makes sure we are asking about a fairly major characters; not someone who is mentioned only once.

chapter_characters = []
for i in range(len(chapter_persons)):
    for x in people:
        if chapter_persons.index[i] == x:
            chapter_characters.append(chapter_persons.index[i])
        else:
            pass    

In [11]:
print(book)
print("Chapter", chapter, ": Questions for Discussion")


#This accounts for if the book has a narrator or not:
if narrator == "":
    narrator = chapter_characters[0]
    i = 1
else:
    i = 0

#This will print up to three character questions per chapter
if len(chapter_characters) == 0:
    pass
elif len(chapter_characters) == 1:
    print("What did", narrator, "talk about with", chapter_characters[i], "?")
elif len(chapter_characters) == 2:
    print("What did", narrator, "talk about with", chapter_characters[i], "?")
    print("What did", narrator, "and", chapter_characters[i+1], "discuss?")
else:
    print("What did", narrator, "talk about with", chapter_characters[i], "?")
    print("What did", narrator, "and", chapter_characters[i+1], "discuss?")
    print("What did", narrator, "say to", chapter_characters[i+2], "?")
    
# We'd print these questions, any vocab words, etc, to a pdf that the user could then print out.

The Best Bad Luck I Ever Had
Chapter 33 : Questions for Discussion
What did Dit talk about with Raymond ?
What did Dit and Emma discuss?
What did Dit say to Pooley ?


In [12]:
# Need to figure out how to deal with characters with prefixes "Mrs. Walker" vs. "Walker"
#We'd write these questions to a pdf